# 데이터 전처리

시설물 데이터의 경우 노원과 인접구 데이터만 갖도록 가공, 포함범위 : 노원 도봉 중랑 성북

라이브러리 불러오기

In [8]:
import numpy as np
import pandas as pd

상수 지정

In [9]:
gu_list=["노원구", "도봉구", "중랑구", "성북구"]
gu_code=[3100000, 3090000, 3060000, 3070000]
start_path = "전처리필요데이터/"
end_path = "전처리완료데이터/"
encoding = 'cp949'

### 지오서비스로 받아온 장소 가공

경찰서 목록 불러오기

In [10]:
police = pd.read_csv(end_path+"police.csv", encoding=encoding)
police = {police.loc[i, "이름"]: police.loc[i, ["위도", "경도"]].values.astype(np.float64) for i in police.index}
police

{'당현지구대': array([ 37.64201519, 127.0683536 ]),
 '노원역지구대': array([ 37.65500523, 127.0669163 ]),
 '화랑지구대': array([ 37.62256592, 127.0745976 ]),
 '불암지구대': array([ 37.65243504, 127.0773404 ]),
 '월계지구대': array([ 37.63368623, 127.0592911 ]),
 '마들지구대': array([ 37.66458702, 127.0636189 ]),
 '당고개파출소': array([ 37.67147105, 127.0778527 ]),
 '상계1파출소': array([ 37.67968126, 127.0550472 ])}

가공

In [16]:
for k in police.keys():
    spath = start_path+f'police/{k}/결과주소.csv'
    epath = end_path+f'police/{k}/{k}_sampled_addr.csv'
    temp = pd.read_csv(spath, encoding=encoding)
    #temp = temp.drop(columns=['_PARCEL_DT', '_ROAD_DT'])
    temp.columns=["위도", "경도", "지번주소", "도로명주소"]
    temp.to_csv(epath, index=False, encoding=encoding)

### 서울시 장소별 범죄발생건수 가공

장소끼리 건수 더한 후, 확률로 변환, 쓰기쉽게 전치

In [ ]:
seoul_crime = pd.read_csv(start_path+"seoul_crime.csv", encoding=encoding)
seoul_crime

In [ ]:
temp = seoul_crime.groupby("장소")
temp = temp.sum()
temp /= temp.sum()
temp.columns=["확률"]
temp = pd.DataFrame(temp.T, columns=temp.index)
temp

유사한 분류 묶은후, 저장

In [ ]:
processed_crime = pd.DataFrame()
processed_crime["공업"] = temp[["공사장광산", "공장", "창고"]].sum(axis=1)
processed_crime["공원"] = temp[["유원지", "공지"]].sum(axis=1)
processed_crime["교통"] = temp[["기타교통수단내", "지하철", "역,대합실"]].sum(axis=1)
processed_crime["기타"] = temp[["구금장소", "고속도로", "기타", "부대"]].sum(axis=1)
processed_crime["사무"] = temp[["금융기관", "사무실"]].sum(axis=1)
processed_crime["숙박"] = temp[["숙박업소목욕탕"]].sum(axis=1)
processed_crime["상업"] = temp[["시장,노점", "상점", "흥행장"]].sum(axis=1)
processed_crime["아파트"] = temp[["아파트,연립다세대"]].sum(axis=1)
processed_crime["야외"] = temp[["노상", "산야"]].sum(axis=1)
processed_crime["유흥"] = temp[["유흥접객업소"]].sum(axis=1)
processed_crime["의료"] = temp[["의료기관"]].sum(axis=1)
processed_crime["종교"] = temp[["종교기관"]].sum(axis=1)
processed_crime["주택"] = temp[["단독주택"]].sum(axis=1)
processed_crime["학교"] = temp[["학교"]].sum(axis=1)
processed_crime

In [ ]:
processed_crime.to_csv(end_path+"processed_crime.csv", index=False,  encoding=encoding)

### 서울시 cctv가공

In [ ]:
seoul_cctv = pd.read_csv(start_path+"seoul_cctv.csv", encoding=encoding)
seoul_cctv

In [ ]:
processed_cctv = pd.DataFrame(columns=["위도", "경도"], dtype=np.float64)
for i in gu_list:
    temp = seoul_cctv.loc[seoul_cctv["자치구"]==i,["위도", "경도"]]
    processed_cctv = pd.concat([processed_cctv, temp])
processed_cctv

In [ ]:
processed_cctv.to_csv(end_path+"processed_cctv.csv", index=False,  encoding=encoding)

### 서울시 안심귀가길 시설물 가공

한 열에 point(경도 위도)형식으로 저장 -> 위경도 추출

In [ ]:
seoul_ansim = pd.read_csv(start_path+"seoul_ansim.csv", encoding=encoding)
seoul_ansim

In [ ]:
processed_ansim = pd.DataFrame()
for i in gu_list:
    processed_ansim = pd.concat([ processed_ansim, seoul_ansim.loc[seoul_ansim["시군구명"] == "서울특별시 "+i, "포인트 wkt"] ])

def word_to_EW(word):
    temp = word.split("(")
    temp = temp[-1].split(" ")
    return temp[0]
def word_to_NS(word):
    temp = word.split("(")
    temp = temp[-1].split(" ")
    return temp[1][:-1]

processed_ansim["위도"] = processed_ansim[0].apply(word_to_NS)
processed_ansim["경도"] = processed_ansim[0].apply(word_to_EW)
processed_ansim = processed_ansim.drop(columns=[0]).reset_index(drop=True)
processed_ansim

In [ ]:
processed_ansim.to_csv(end_path+"processed_ansim.csv", index=False, encoding=encoding)

### 서울시 보안등 가공

하려했으나 중랑구, 성북구는 데이터가 없어 제외

In [ ]:
nowon_light = pd.read_csv(start_path+"nowon_light.csv", encoding=encoding)
nowon_light

In [ ]:
dobong_light = pd.read_csv(start_path+"dobong_light.csv", encoding=encoding)
dobong_light

In [ ]:
processed_light = pd.concat([nowon_light, dobong_light]).reset_index(drop=True)
processed_light

In [ ]:
processed_light.to_csv(end_path+"processed_light.csv", index=False,  encoding=encoding)

### 서울시 유흥주점 가공

범위내 영업중 장소 골라냄

In [ ]:
seoul_salon = pd.read_csv(start_path+"seoul_salon.csv", encoding=encoding)
seoul_salon

In [ ]:
seoul_salon = seoul_salon.loc[seoul_salon["상세영업상태명"]=="영업", :]
temp_salon = pd.DataFrame()
for i in gu_code:
    temp_salon = pd.concat([ temp_salon, seoul_salon.loc[seoul_salon["개방자치단체코드"] == i, ["지번주소", "도로명주소"]] ])
temp_salon = temp_salon.reset_index(drop=True)
temp_salon

해당데이터는 주소로만 이뤄져있으므로 위경도로 변환해야함

https://www.geoservice.co.kr/ 이용

In [ ]:
temp_salon.to_csv(start_path+"temp_salon.csv", index=False,  encoding=encoding)

위경도 변환된 데이터 확인

In [ ]:
processed_salon = pd.read_csv(end_path+"processed_salon.csv", encoding=encoding)
processed_salon